# Projeto Final

Para esse projeto foi considerado a base de dados em Português do Brasil de reviews disponíveis no IMDB.

Esse site contém uma ampla variedade de catálogos de filmes em que as pessoas cadastradas podem realizar comentários com suas opiniões.

Essa base de dados está disponível em: 
* https://huggingface.co/datasets/celsowm/imdb-reviews-pt-br

## Fazendo Importações Necessárias

Para execução do código presente nesse Notebook, será necessário a realização dos imports a seguir. 

Essa linha deve sempre ser executada primeiro.

In [2]:
import os

os.environ['TOKENIZERS_PARALLELISM'] = "false"

import snorkel
import pandas as pd
from snorkel.labeling import (LFAnalysis, PandasLFApplier,filter_unlabeled_dataframe, labeling_function)
from snorkel.labeling.model.label_model import LabelModel
from snorkel.utils import probs_to_preds

snorkel.__version__

'0.10.0'

### Carregando o dataset

In [3]:
df = pd.read_parquet('datasets/train-00000-of-00001-34e817339f28df45.parquet')
df.head()

,id,texto,sentimento
0,1,"Mais uma vez, o Sr. Costner arrumou um filme p...",0
1,2,Este é um exemplo do motivo pelo qual a maiori...,0
2,3,"Primeiro de tudo eu odeio esses raps imbecis, ...",0
3,4,Nem mesmo os Beatles puderam escrever músicas ...,0
4,5,Filmes de fotos de latão não é uma palavra apr...,0


In [4]:
# Indicando o random_state apenas para conseguir refazer com os mesmos resultados.
for row in df.sample(50, random_state=1).itertuples():
    print(f"{row.texto} - {row.id}")
    print('\n')

Assassinatos estão ocorrendo em uma cidade do deserto do Texas. Quem é responsável? Ligeiras novidades de mistério e tensões raciais este último realmente não se encaixa, mas de outra forma estritamente para os fãs de slasher, que irão apreciar a nesga e nudez, que são dois elementos convencionais para estes filmes.Dana Kimmell de Sexta-feira 13ª infâmia estrelas como o malcriado adolescente quase-detetive.1 / 2 da MPAA: Pontuação: R por violência e nesga, nudez e alguma linguagem. - 31656


Prós: NothingCons: EverythingPlot resumo: Uma repórter fêmea corre para um caroneiro que conta suas histórias sobre as mortes de pessoas que foram mortas por zumbis. Revisão: Nunca na minha vida eu me deparei com um filme tão ruim quanto The Zombie Chronicles. Filmado em um orçamento do que parece ser cerca de 20 dólares, o TZC é um filme de terror completamente horrível que conta com atores esquecidos e esquecidos que não poderiam agir para salvar suas vidas, o que é mais grosseiro do que assustad

## Transformando o texto para letras minúsculas

Essa transformação facilitará os posteriores processamentos que serão realizados.

In [5]:
df["texto"] = df["texto"].str.lower()
df

,id,texto,sentimento
0,1,"mais uma vez, o sr. costner arrumou um filme p...",0
1,2,este é um exemplo do motivo pelo qual a maiori...,0
2,3,"primeiro de tudo eu odeio esses raps imbecis, ...",0
3,4,nem mesmo os beatles puderam escrever músicas ...,0
4,5,filmes de fotos de latão não é uma palavra apr...,0
...,...,...,...
49454,49456,"como a média de votos era muito baixa, e o fat...",1
49455,49457,o enredo teve algumas reviravoltas infelizes e...,1
49456,49458,estou espantado com a forma como este filme e ...,1
49457,49459,a christmas together realmente veio antes do m...,1


Também precisaremos remover a coluna sentimento. Nos processamentos que iremos fazer, não precisamos dessa coluna. Assim como vai ser interessante embaralharmos as linhas.

In [10]:
df_reviews = df.drop("sentimento", axis = 1)
df_reviews 

,id,texto
0,1,"mais uma vez, o sr. costner arrumou um filme p..."
1,2,este é um exemplo do motivo pelo qual a maiori...
2,3,"primeiro de tudo eu odeio esses raps imbecis, ..."
3,4,nem mesmo os beatles puderam escrever músicas ...
4,5,filmes de fotos de latão não é uma palavra apr...
...,...,...
49454,49456,"como a média de votos era muito baixa, e o fat..."
49455,49457,o enredo teve algumas reviravoltas infelizes e...
49456,49458,estou espantado com a forma como este filme e ...
49457,49459,a christmas together realmente veio antes do m...


In [7]:
df

,id,texto,sentimento
0,1,"mais uma vez, o sr. costner arrumou um filme p...",0
1,2,este é um exemplo do motivo pelo qual a maiori...,0
2,3,"primeiro de tudo eu odeio esses raps imbecis, ...",0
3,4,nem mesmo os beatles puderam escrever músicas ...,0
4,5,filmes de fotos de latão não é uma palavra apr...,0
...,...,...,...
49454,49456,"como a média de votos era muito baixa, e o fat...",1
49455,49457,o enredo teve algumas reviravoltas infelizes e...,1
49456,49458,estou espantado com a forma como este filme e ...,1
49457,49459,a christmas together realmente veio antes do m...,1


O próximo trecho de código, será um conjunto de funções que servirá para remover e tratar ainda mais as informações do texto, removendo aquilo que não é necessário.

Podemos citar mais de um espaço em branco, acentuações e pontuações.

In [11]:
import re
import unicodedata

def remove_espacos_excessivos(text: str) -> str:
    # \s+ matches one or more whitespace characters
    # ' ' replaces the matched string with a single space
    # strip() removes leading and trailing spaces
    return re.sub(r'\s+', ' ', text).strip() 

def remove_caracteres_repetidos_nao_palavras(text: str) -> str:
    # \W matches any non-word character (equivalent to [^a-zA-Z0-9_ ])
    # \1+ matches one or more occurrences of the same character
    # r'\1' replaces the matched string with a single occurrence of the character
    # strip() removes leading and trailing spaces
    return re.sub(r'(\W)\1+', r'\1', text).strip()

def remove_letras_repetidas(text: str, n_repeat: int = 4) -> str:
    # ([a-z]) matches a lowercase letter
    # \1{'+str(n_repeat)+',}' matches n_repeat or more occurrences of the same letter
    # r'\1' replaces the matched string with a single occurrence of the letter
    # strip() removes leading and trailing spaces
    return re.sub(r'([a-z])\1{'+str(n_repeat)+',}', r'\1', text).strip()

def remove_acentos(text: str) -> str:
    # normalize() converts the text to the NFKD form
    # encode() converts the text to bytes, ignoring non-ASCII characters
    # decode() converts the bytes back to a string, ignoring errors
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

In [ ]:
from sklearn.pipeline import Pipeline 
from sklearn.preprocessing import FunctionTransformer 


pipeline_clean_text = Pipeline([
    ('remove_acentos', FunctionTransformer(remove_acentos)),
    ('remove_espacos_excessivos', FunctionTransformer(remove_espacos_excessivos)),
    ('remove_letras_repetidas', FunctionTransformer(remove_letras_repetidas)),
    ('remove_caracteres_repetidos_nao_palavras', FunctionTransformer(remove_caracteres_repetidos_nao_palavras)),
])